### Gamma Categorization

- Design Patters are typically split into three categories
- This is called _Gamma Categorization_ after Eric Gamma, one of GoF authors

**Creational Patterns**
- Deals with creation (construction) of objects.
- Explicit (constructor) vs. implicit (DI, reflection, etc.)
- Wholesale (single statement) vs. piecewise (step-by-step)

**Structural Patterns**
- Concerned with the structure (eg. class members)
- Many patterns are wrappers that mimic the underlying class interface
- Stress the importance of good API design

**Behavioral Pattern**
- They are all different; no central theme

-----

# Builder Design Pattern
When construction gets a little bit too complicated

- Some objects are simple and can be created in a single initilizer call
- Other objects requires a lot of ceremony to create
- Having an object with 10 initializer argument is not productive
- Instead, opt for piecewise construction
- Builder provides an API for constructing an object step-by-step
----

Def: When piecewise object construction is complicated, provide an API for doing it succinctly

Why do we need it?
- Builder is required when you have some sort of construction of object, so not just constructing an object in a single statement, but actually taking several statements to construct something. 

In [2]:
text = "Hello"
parts = ['<p>', text, '</p>']
print("".join(parts))

<p>Hello</p>


The above example is quite simple, it just took 2 lines to generate

Now, let's say you given bunch of words and you are suppose to make an HTML list out of it

In [4]:
words = ['hello', 'world']
parts = ['<ul>']
for w in words:
    parts.append(f"    <li>{w}</li>")
parts.append('</ul>')

print("\n".join(parts))

<ul>
    <li>hello</li>
    <li>world</li>
</ul>


Now, in the above example, you have to write 5 lines of code to generate the html tag.
Now, what we want is if someone is using any html tag, we want it to be generated with proper closing tag

In [7]:
class HtmlElement:
    indent_size = 2
    
    def __init__(self, name='', text=''):
        self.text = text
        self.name = name
        self.elements = []
        
    def __str(self, indent):
        lines = []
        i = ' ' * (indent * self.indent_size)
        lines.append(f"{i}<{self.name}>")
        
        if self.text:
            i1 = ' ' * ((indent + 1) * self.indent_size)
            lines.append(f"{i1}{self.text}")
            
        for e in self.elements:
            lines.append(e.__str(indent + 1))
            
        lines.append(f"{i}</{self.name}>")
        return "\n".join(lines)
    
    def __str__(self):
        return self.__str(0)
    
class HtmlBuilder:
    def __init__(self, root_name):
        self.root_name = root_name
        self.__root = HtmlElement(name=root_name)
        
    def __str__(self):
        return str(self.__root)
    
    def add_child(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        
    def add_child_fluent(self, child_name, child_text):
        self.__root.elements.append(
            HtmlElement(child_name, child_text)
        )
        return self
        
builder = HtmlBuilder("ul")
builder.add_child("li", "hello")
builder.add_child("li", "world")

print("ORDINARY BUILDER")
print(builder)

ORDINARY BUILDER
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


In [9]:
builder = HtmlBuilder("ul")
builder.add_child_fluent("li", "hello").add_child_fluent("li", "world")

print("ORDINARY BUILDER")
print(builder)

ORDINARY BUILDER
<ul>
  <li>
    hello
  </li>
  <li>
    world
  </li>
</ul>


### Builder Facets

Now we'll see the complications of the builder design pattern, Sometimes you have an object that is so complicated that you need more than one builder to do it. So, how can we do that. 

In [11]:
class Person:
    def __init__(self):
        
        # address
        self.street_address = None
        self.postcode = None
        self.city = None
        
        # employment
        self.company_name = None
        self.position = None
        self.annual_income = None
    
    def __str__(self) -> str:
        return (
            f"Address: {self.street_address}, {self.postcode}, {self.city} "
            f"Employed at: {self.company_name} as a {self.position} earning {self.annual_income}"
        )
    
class PersonBuilder:
    def __init__(self, person=Person()):
        self.person = person
        
    @property
    def works(self):
        return PersonJobBuilder(self.person)
    
    @property
    def lives(self):
        return PersonAddressBuilder(self.person)
        
    def build(self):
        return self.person

    
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
        
    def at(self, company_name):
        self.person.company_name = company_name
        return self
    
    def as_a(self, position):
        self.person.position = position
        return self
    
    def earning(self, annual_income):
        self.person.annual_income = annual_income
        return self
    
    
class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
        
    def at(self, street_address):
        self.person.street_address = street_address
        return self
    
    def with_postcode(self, postcode):
        self.person.postcode = postcode
        return self
    
    def in_city(self, city):
        self.person.city = city
        return self

We now constructed an interface where you can use two builders `PersonJobBuilder` and `PersonAddressBuilder` interchangabely through a common API because they both inherit from person builder

In [17]:
pb = PersonBuilder()
person = pb\
        .lives\
            .at('123 London Road')\
            .in_city("London")\
            .with_postcode("SW12BC")\
        .works\
            .at("Fabrikan")\
            .as_a("software engineer")\
            .earning("50000")\
        .build()

In [18]:
person

In [20]:
print(str(person))

Address: 123 London Road, SW12BC, London Employed at: Fabrikan as a software engineer earning 50000


### Builder Inheritance

One of the things you might have noticed is that, it violates the open-closed principle, because whenever you have a new sub-builder you have to add it to the PersonBuilder, so there is an alternative approach to this entire story is to simply use inheritance.

In [22]:
class Person:
    def __init__(self):
        self.name = None
        self.position = None
        self.date_of_birth = None
        
    def __str__(self):
        return f"{self.name} | {self.position} | {self.date_of_birth}"
    
    @staticmethod
    def new():
        return PersonBuilder()
    
class PersonBuilder:
    def __init__(self):
        self.person = Person()
        
    def build(self):
        return self.person
    
class PersonInfoBulder(PersonBuilder):
    def called(self, name):
        self.person.name = name
        return self
    
class PersonJobBuilder(PersonInfoBulder):
    def works_as_a(self, position):
        self.person.position = position
        return self
    
class PersonBirthDateBuilder(PersonJobBuilder):
    def born(self, date_of_birth):
        self.person.date_of_birth = date_of_birth
        return self
    
pb = PersonBirthDateBuilder()
me = pb\
        .called("Dmitri")\
        .works_as_a("Quant")\
        .born("1/1/1980")\
        .build()
print(str(me))

Dmitri | Quant | 1/1/1980


## Summary
- A builder is a separate component for building an object
- Can either give builder an initializer or return it via a static function
- To make builder fluent, return self
- Different facets of an object can built with different builders working in tandem via a base class